# AI Model Training & Export for Prosthetic Hand

This notebook trains an AI model on your ESP32 data and **exports it as C++ code (`model.h`)**.

**Workflow:**
1.  **Collect Data:** Use `Prosthetic_Hand_Data_Collection.ino` to create `prosthetic_data.csv`.
2.  **Upload CSV:** Upload `prosthetic_data.csv` to this notebook.
3.  **Train Model:** Run cells to train a Random Forest model.
4.  **Export Model:** Run the final cells to convert the model to C++ code.
5.  **Deploy:** Copy the C++ code into a new `model.h` file in your Arduino project and upload `Prosthetic_Hand_AI_Control.ino`.

## Step 1: Install Libraries

We need `micromlgen` to convert our `scikit-learn` model to C++.

In [ ]:
!pip install pandas matplotlib seaborn scikit-learn micromlgen

## Step 2: Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from micromlgen import port

## Step 3: Load and Explore Data

Upload your `prosthetic_data.csv` file using the 'Files' tab on the left.

In [ ]:
file_path = 'prosthetic_data.csv'
try:
    df = pd.read_csv(file_path)
    print("Data loaded successfully!")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: '{file_path}' not found.")
    print("Please upload your CSV file to Colab.")

if 'df' in locals():
    print("\nFirst 5 rows of data:")
    print(df.head())

    print("\nData summary:")
    print(df.describe())

### Visualize the Relationship (Optional)

In [ ]:
if 'df' in locals():
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='grip_avg', y='servo_pos', alpha=0.5)
    plt.title('Grip Pressure vs. Servo Position')
    plt.xlabel('Average Grip Pressure (0-4095)')
    plt.ylabel('Servo Pulse (150-600)')
    plt.grid(True)
    plt.show()

## Step 4: Define Features (X) and Target (y)

**IMPORTANT:** The order of features here **must** match the order in the `Prosthetic_Hand_AI_Control.ino` file.

In [ ]:
if 'df' in locals():
    # Features: The inputs for the AI
    feature_names = ['fsr1', 'fsr2', 'accelX', 'accelY', 'accelZ']
    
    # Target: The output we want the AI to predict
    target_name = 'servo_pos'

    X = df[feature_names]
    y = df[target_name]

    print("Features (X) shape:", X.shape)
    print("Target (y) shape:", y.shape)

## Step 5: Split Data into Training and Testing Sets

In [ ]:
if 'X' in locals():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training data size: {X_train.shape[0]} samples")
    print(f"Testing data size: {X_test.shape[0]} samples")

## Step 6: Train the AI Model

We use a `RandomForestRegressor`. We use `n_estimators=10` and `max_depth=5` to keep the model **small** so it fits on the ESP32.

In [ ]:
if 'X_train' in locals():
    # Initialize a *small* model for the ESP32
    model = RandomForestRegressor(
        n_estimators=10,  # Use fewer trees
        max_depth=5,      # Limit tree depth
        random_state=42
    )

    print("Training the model...")
    model.fit(X_train, y_train)
    print("Model training complete!")

## Step 7: Evaluate the Model

In [ ]:
if 'model' in locals():
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("--- Model Evaluation ---")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R-squared (R2) Score: {r2:.2f}")
    print("\n(R2 Score > 0.9 is excellent. If it's low, collect more/better data)")

    # Plot predictions vs actual values
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red', lw=2)
    plt.xlabel('Actual Servo Position')
    plt.ylabel('Predicted Servo Position')
    plt.title('Model Predictions vs. Actual Values')
    plt.grid(True)
    plt.show()

## Step 8: EXPORT THE MODEL TO C++

This is the final step. We will use `micromlgen` to convert our trained `model` into C++ code that you can copy into your Arduino `model.h` file.

In [ ]:
if 'model' in locals():
    print("--- EXPORTING MODEL TO C++ CODE ---\n")
    
    # Convert the model
    cpp_code = port(model, classname='ProstheticHandModel', features=feature_names)
    
    # Print the C++ code
    # 1. Click the 'Copy' icon that appears in the top-right of the output block.
    # 2. Go to your Arduino IDE.
    # 3. Create a new tab and name it 'model.h'.
    # 4. Paste this ENTIRE code into that 'model.h' file.
    print(cpp_code)
    
    print("\n--- COPY THE CODE ABOVE THIS LINE --- ")
    print("Paste it into 'model.h' in your Arduino sketch folder.")